<a href="https://colab.research.google.com/github/DougJ/DougJ.github.io/blob/master/Tensor_From_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create tensors from a CSV file. 
1) Load CSV file as dataframe  
2) Create two unique data frames for single and multi class
3) Export to two unique csv files  
4) Create directories based on unique single class  
5) Copy images from directories into single class target directory   
6) Consider removing or compressing original image file  directories   
7) Explore data, identify if need to adjust prior to tensors   
8) Create tensors   

### TO DISCUSS
Covid images, or ignore for now?  
Doing:
   Cardiomegaly not Cardiomegaly  tensors? (meaning everything not Cariomegaly)  
   Healthy not Healthy (meaning everything not Healthy)  
If doing this, do we balance?  

Or

Doing:
Tensor for each individual class?  
If doing this, do we balance?  

Do we train on imagenet
Then train on healthy (lot of images, can create a bonus training set. Do we need to train on pnumothorax?)  
Then train on unhealthy  

Do we want to also pull from Stanford site?  

In [0]:
import pandas as pd
import os
import shutil

### 1) Load CSV file as dataframe  

In [281]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [282]:
cd /content/drive/'My Drive'/images/'NIH chest X ray images'/

/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images


In [283]:
pwd

'/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images'

In [284]:
ls

 Atelectasis/                      images_001/   images_012/
 Cardiomegaly/                     images_002/   Infiltration/
 Consolidation/                    images_003/   Mass/
 Data_Entry_2017.csv               images_004/   Nodule/
 Data_Entry_2017_multi_class.csv   images_005/  'No Finding'/
 Data_Entry_2017_one_class.csv     images_006/   Pleural_Thickening/
 Edema/                            images_007/   Pneumonia/
 Effusion/                         images_008/   Pneumothorax/
 Emphysema/                        images_009/   README_CHESTXRAY.pdf
 Fibrosis/                         images_010/
 Hernia/                           images_011/


In [285]:
df = pd.DataFrame()
df = pd.read_csv('Data_Entry_2017.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


### 2) Create two unique data frames for single and multi class

In [0]:
df_pipe = pd.DataFrame(columns=df.columns)
df_noPipe = pd.DataFrame(columns=df.columns)

In [0]:
cols_to_copy = ['Image Index', 'Finding Labels', 'View Position']

# SPECIFIC COLUMNS
#df_one_class = df.loc[~df['Finding Labels'].str.contains(r'\|'), cols_to_copy].copy()
#df_multi_class = df.loc[df['Finding Labels'].str.contains(r'\|'), cols_to_copy].copy()

# ALL COLUMNS
df_one_class = df.loc[~df['Finding Labels'].str.contains(r'\|'), :].copy()
df_multi_class = df.loc[df['Finding Labels'].str.contains(r'\|'), :].copy()

In [0]:
df_one_class = positive_df.loc[(positive_df['view']== 'PA')|
                                 (positive_df['view']== 'AP')|
                                 (positive_df['view']== 'AP Supine'), :]
print(pos_good_views.shape)
pos_good_views.head()

(226, 6)


,filename,patientid,finding,view,clinical_notes,url
0,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,2,COVID-19,PA,infiltrate in the upper lobe of the left lung,https://www.nejm.org/doi/full/10.1056/NEJMc200...
1,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,2,COVID-19,PA,progressive infiltrate and consolidation,https://www.nejm.org/doi/full/10.1056/NEJMc200...
2,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,2,COVID-19,PA,progressive infiltrate and consolidation,https://www.nejm.org/doi/full/10.1056/NEJMc200...
3,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,2,COVID-19,PA,progressive infiltrate and consolidation,https://www.nejm.org/doi/full/10.1056/NEJMc200...
4,nejmc2001573_f1a.jpeg,4,COVID-19,PA,diffuse infiltrates in the bilateral lower lungs,https://www.nejm.org/doi/full/10.1056/NEJMc200...


### 3) Export to two unique csv files 

In [0]:
df_one_class.to_csv('Data_Entry_2017_one_class.csv', sep='\t', encoding='utf-8')

In [0]:
df_multi_class.to_csv('Data_Entry_2017_multi_class.csv', sep='\t', encoding='utf-8')

### 4) Create directories based on unique single class  


In [312]:
df_multi_class.head(1)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN


In [313]:
df_one_class.head(1)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN


In [318]:
df_one_class.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Image Index,91324,91324,00022481_006.png,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Finding Labels,91324,15,No Finding,60361,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Follow-up #,91324,NaN,NaN,NaN,7.46394,14.2027,0,0,2,8,182
Patient ID,91324,NaN,NaN,NaN,14181.9,8449.72,1,7034,13729.5,20624,30805
Patient Age,91324,NaN,NaN,NaN,46.5221,16.7689,1,34,48,59,414
Patient Gender,91324,2,M,51343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
View Position,91324,2,PA,57130,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OriginalImage[Width,91324,NaN,NaN,NaN,2638.68,343.052,1143,2500,2504,2992,3827
Height],91324,NaN,NaN,NaN,2490.89,403.309,966,2048,2541,2991,4715
OriginalImagePixelSpacing[x,91324,NaN,NaN,NaN,0.155802,0.0162296,0.115,0.143,0.143,0.168,0.1988


In [315]:
df_multi_class.describe()

,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
count,20796.000000,20796.000000,20796.000000,20796.000000,20796.000000,20796.000000,20796.000000,0.0
mean,13.447394,15068.495672,48.567609,2678.590787,2466.900414,0.154976,0.154976,NaN
std,19.097351,8160.584950,17.049257,331.258544,391.590376,0.015910,0.015910,NaN
min,0.000000,1.000000,1.000000,1215.000000,1077.000000,0.115000,0.115000,NaN
25%,2.000000,8841.000000,36.000000,2500.000000,2048.000000,0.143000,0.143000,NaN
50%,7.000000,14888.000000,51.000000,2544.000000,2544.000000,0.143000,0.143000,NaN
75%,17.000000,20852.000000,61.000000,2992.000000,2991.000000,0.168000,0.168000,NaN
max,183.000000,30801.000000,412.000000,3550.000000,3166.000000,0.198800,0.198800,NaN


In [294]:
df_one_class['Finding Labels'].value_counts()

No Finding            60361
Infiltration           9547
Atelectasis            4215
Effusion               3955
Nodule                 2705
Pneumothorax           2194
Mass                   2139
Consolidation          1310
Pleural_Thickening     1126
Cardiomegaly           1093
Emphysema               892
Fibrosis                727
Edema                   628
Pneumonia               322
Hernia                  110
Name: Finding Labels, dtype: int64

In [319]:
df_one_class['View Position'].value_counts()

PA    57130
AP    34194
Name: View Position, dtype: int64

In [295]:
print("# unique = {}\n".format(df_multi_class['Finding Labels'].nunique()))
df_multi_class['Finding Labels'].value_counts()

# unique = 821



Effusion|Infiltration                                   1603
Atelectasis|Infiltration                                1350
Atelectasis|Effusion                                    1165
Infiltration|Nodule                                      829
Atelectasis|Effusion|Infiltration                        737
                                                        ... 
Atelectasis|Consolidation|Nodule|Pneumonia                 1
Fibrosis|Hernia|Infiltration                               1
Edema|Infiltration|Pneumothorax                            1
Atelectasis|Fibrosis|Infiltration|Pleural_Thickening       1
Edema|Pneumonia|Infiltration                               1
Name: Finding Labels, Length: 821, dtype: int64

In [309]:
# SPLIT EXAMPLE FOR 'Finding Labels' BASED ON '|' SO CAN USE DOWN THE ROAD 
count = 0
for i, row in df_multi_class.iterrows():
  if count > 4:
    break
  str = row['Finding Labels']
  split_str = str.split('|')
  print(split_str)
  count+=1

['Cardiomegaly', 'Emphysema']
['Cardiomegaly', 'Effusion']
['Hernia', 'Infiltration']
['Mass', 'Nodule']
['Effusion', 'Infiltration']


In [310]:
# LOOK AT LABELS FOR IMAGES WITH JUST ONE CLASS IN 'Finding Labels'
class_list = df_one_class['Finding Labels'].unique().tolist()
class_list

['Cardiomegaly',
 'No Finding',
 'Hernia',
 'Infiltration',
 'Nodule',
 'Emphysema',
 'Effusion',
 'Atelectasis',
 'Pleural_Thickening',
 'Pneumothorax',
 'Mass',
 'Fibrosis',
 'Consolidation',
 'Edema',
 'Pneumonia']

In [298]:
# CHANGE DIRECTORY TO PLACE WHERE WANT TO MAKE DIRECTORES
cd /content/drive/'My Drive'/images/'NIH chest X ray images'/

/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images


In [299]:
# MAKE DIRECTORIES
for dirName in class_list:
  if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

Directory  Pneumonia  already exists


In [300]:
# VERIFY DIRECTORIES EXIST
ls

 Atelectasis/                      images_001/   images_012/
 Cardiomegaly/                     images_002/   Infiltration/
 Consolidation/                    images_003/   Mass/
 Data_Entry_2017.csv               images_004/   Nodule/
 Data_Entry_2017_multi_class.csv   images_005/  'No Finding'/
 Data_Entry_2017_one_class.csv     images_006/   Pleural_Thickening/
 Edema/                            images_007/   Pneumonia/
 Effusion/                         images_008/   Pneumothorax/
 Emphysema/                        images_009/   README_CHESTXRAY.pdf
 Fibrosis/                         images_010/
 Hernia/                           images_011/


### 5) Copy images from directories into single class target directory  

In [0]:
#ls /content/drive/'My Drive'/images/'NIH chest X ray images'/images_001/images/

In [0]:
# VARIABLES FOR COPY
files = ['00000001_000.png', '00000456_000.png', '00000877_026.png', '00000001_001.png', '00000456_001.png']
source = '/content/drive/My Drive/images/NIH chest X ray images/images_001/images/'
target = '/content/drive/My Drive/images/NIH chest X ray images/No Finding/'

In [303]:
# NEED TO BE IN CORRECT BASE DIRECTORY TO COPY FROM, NOT PASSING THIS INTO COPY FUNCTION, ONLY PASS TARGET
cd '/content/drive/My Drive/images/NIH chest X ray images/images_001/images/'

/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images/images_001/images


In [306]:
# COPY IMAGES FROM SOURCE TO TARGET DESTINATION
for file in files:
  if not os.path.exists(target):
    print("{} does not exist".format(target))
  else:
    #shutil.copy(file, target)
    print("copied {} to: {}".format(file, target))

copied 00000001_000.png to: /content/drive/My Drive/images/NIH chest X ray images/No Finding/
copied 00000456_000.png to: /content/drive/My Drive/images/NIH chest X ray images/No Finding/
copied 00000877_026.png to: /content/drive/My Drive/images/NIH chest X ray images/No Finding/
copied 00000001_001.png to: /content/drive/My Drive/images/NIH chest X ray images/No Finding/
copied 00000456_001.png to: /content/drive/My Drive/images/NIH chest X ray images/No Finding/


In [307]:
# VERIFY COPIED CORRECTLY
ls '/content/drive/My Drive/images/NIH chest X ray images/No Finding/'

00000001_000.png  00000456_000.png  00000877_026.png
00000001_001.png  00000456_001.png


### 6) Consider removing or compressing original image file directories  


### 7) Explore data, identify if need to adjust prior to tensors  

### 8) Create tensors


In [320]:
cd '/content/drive/My Drive/images/NIH chest X ray images/No Finding/'

/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images/No Finding


In [0]:
# CREATE A LIST OF ALL FILES IN FOLDER
from os import listdir
from os.path import isfile, join
mypath = '/content/drive/My Drive/images/NIH chest X ray images/No Finding/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [323]:
for item in onlyfiles:
  print(item)

00000001_000.png
00000456_000.png
00000877_026.png
00000001_001.png
00000456_001.png


## !!! MAKE THIS A FUNCTION

In [0]:
import cv2

nrows = 224
ncolumns = 224
channels = 3
X_pos = []

# FOR IMAGE IN DIRECTORY
for item in onlyfiles:
  
  # cv2.IMREAD_COLOR reads as RGB colors but no transparency channel. This is the default value 
  # cv2.IMREAD_GRAYSCALE reads as grey image. If color image, grey value calculated by taking the average of color channels.
  # cv2.IMREAD_UNCHANGED reads the image as is from the source.
  b = cv2.imread(item, cv2.IMREAD_COLOR)
  
  # interpolation methods are cv.INTER_AREA for shrinking and cv.INTER_CUBIC(slow) & cv.INTER_LINEAR for zooming. 
  # By default, interpolation method used is cv.INTER_LINEAR for all resizing purposes.
  b = cv2.resize(b,(nrows, ncolumns), interpolation = cv2.INTER_CUBIC)
 
  # ADD TO A LIST 
  X_pos.append(b) 

In [327]:
print(len(X_pos))
#print(X_pos)

5


# !!! stop code here

In [0]:
break

## !!! TEST TRAIN VAL SPLIT ON LIST OF IMAGES

In [0]:
from sklearn.model_selection import train_test_split
#X_train_pos, X_val_pos = train_test_split(X_pos, test_size = .1, random_state = 42)
#X_train_pos, X_test_pos = train_test_split(X_train_pos, test_size = .3, random_state = 42)

## !!! MAKE TENSORS ONCE SPLIT

In [0]:
np.save('X_train', np.array(X_train))
np.save('y_train', np.array(y_train))
np.save('X_test', np.array(X_test))
np.save('y_test', np.array(y_test))
np.save('X_val', np.array(X_val))
np.save('y_val', np.array(y_val))

## !!! GENERATE LABEL LIST OF TWO DIMENSIONS


In [0]:
y_0_train = list(np.zeros(len(X_train_neg)))
y_1_train = list(np.ones(len(X_train_pos)))
y_train = y_0_train + y_1_train

y_0_test = list(np.zeros(len(X_test_neg)))
y_1_test = list(np.ones(len(X_test_pos)))
y_test = y_0_test + y_1_test

y_0_val = list(np.zeros(len(X_val_neg)))
y_1_val = list(np.ones(len(X_val_pos)))
y_val = y_0_val + y_1_val

### 9) Remove directories or zip directories after created tensors to save space.


In [308]:
# IF NEED REMOVE DIRECTORIES CREATED AFTER CREATE TENSOR TO SAVE SPACE
for dirName in class_list:
  if not os.path.exists(dirName):
    print("Directory " , dirName ,  " Does not exist ")
  else:
    #os.rmdir(dirName)
    print("Directory " , dirName ,  " Removed ")

Directory  Cardiomegaly  Does not exist 
Directory  No Finding  Does not exist 
Directory  Hernia  Does not exist 
Directory  Infiltration  Does not exist 
Directory  Nodule  Does not exist 
Directory  Emphysema  Does not exist 
Directory  Effusion  Does not exist 
Directory  Atelectasis  Does not exist 
Directory  Pleural_Thickening  Does not exist 
Directory  Pneumothorax  Does not exist 
Directory  Mass  Does not exist 
Directory  Fibrosis  Does not exist 
Directory  Consolidation  Does not exist 
Directory  Edema  Does not exist 
Directory  Pneumonia  Does not exist 
